In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import backtest
from plotting import plot_fills
from downloader import Downloader, prep_config
from pure_funcs import denumpyize, numpyize, get_template_live_config, candidate_to_live_config, calc_spans, get_template_live_config
from procedures import make_get_ticks_cache, dump_live_config, load_live_config
from time import time
from passivbot import add_argparse_args
from analyze import analyze_fills
import sys
import argparse
import pprint
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [ ]:
plt.rcParams['figure.figsize'] = [29, 18]
pd.set_option('precision', 10)

In [ ]:
# if spans change, new data cache must be made
min_span = 9000
max_span = 160000
n_spans = 3
spans = calc_spans(min_span, max_span, n_spans)
spans

In [ ]:
class Args:
    def __init__(self):
        self.backtest_config_path = 'configs/backtest/default.hjson'
        self.optimize_config_path = 'configs/optimize/default.hjson'
        self.symbol = 'BTSUSDT'
        self.user = 'your_user_name'
        self.start_date = '2020-12-09'
        self.end_date = '2021-06-10'
config = await prep_config(Args())
config['min_span'] = min_span
config['max_span'] = max_span
config['n_spans'] = n_spans
config['spans'] = spans
dl = Downloader(config)
sts = time()
ticks = await dl.get_ticks(True)
prices, buyer_maker, timestamps, emas = make_get_ticks_cache(config, ticks)
data = (prices, buyer_maker, timestamps, emas)
print(f'millis to load {len(ticks)} ticks {(time() - sts) * 1000:.0f}ms')

In [ ]:
ratios = np.c_[prices, emas[:,:-1]] / emas
df = pd.DataFrame({**{'price': data[0], 'buyer_maker': data[1], 'timestamp': data[2]},
                   **{f'MA_{i}': data[3][:,i] for i in range(len(data[3][0]))},
                   **{f'ratio_{i}': ratios[:,i] for i in range(len(data[3][0]))},
                      'MA_band_lower': data[3].min(axis=1), 'MA_band_upper': data[3].max(axis=1)})
df

In [ ]:
df[['MA_band_lower', 'MA_band_upper']].iloc[::100].plot()

In [ ]:
# long stop loss price and shrt ientry price are never lower than upper stop band
# shrt stop loss price and long ientry price are never higher than lower stop band

In [ ]:
# max n days between consecutive occurrences of price crossing lower MA band
df[df.price <= df.MA_band_lower].timestamp.diff().max() / 1000 / 60 / 60 / 24

In [ ]:
# max n days between consecutive occurrences of price crossing upper MA band
df[df.price >= df.MA_band_upper].timestamp.diff().max() / 1000 / 60 / 60 / 24

In [ ]:
bytes_per_mb = 1000000
millis_per_day = 1000 * 60 * 60 * 24
sizebytes = 0
for d in data:
    print(d.dtype)
    sizebytes += sys.getsizeof(d)
bytes_per_tick = sizebytes / len(prices)
print('bytes_per_tick', bytes_per_tick, 'data size in mb', sizebytes  / bytes_per_mb)

n_days = (timestamps[-1] - timestamps[0]) / millis_per_day
ticks_per_day = len(prices) / n_days
bytes_per_day = bytes_per_tick * ticks_per_day
millis_per_tick = n_days / len(prices) * millis_per_day
print('n_days', n_days, 'millis_per_tick', millis_per_tick)

In [ ]:
config['starting_balance'] = 100.0
config['latency_simulation_ms'] = 750
config['maker_fee'] = 0.00018

In [ ]:
template = get_template_live_config(min_span=min_span,
                                    max_span=max_span,
                                    n_spans=n_spans)
template

In [ ]:
# tune it...

#template['shrt']['iprc_const'] = 1.0085
#template['long']['rprc_MAr_coeffs'][0][0] = -0.005

template['long']['rprc_PBr_coeffs'][0][1] = -0.1
template['shrt']['rprc_PBr_coeffs'][0][1] = 0.1
template['long']['rqty_const'] = 0.1
template['shrt']['rqty_const'] = 0.1


# etc

In [ ]:
config_to_test = {**config, **numpyize(template)}
#dump_live_config(config_to_test, 'hand_tuned.json')

In [ ]:
sts = time()
fills, info = backtest(config_to_test, data)
print(f'seconds elapsed {time() - sts:.4f}')
print(f'did finish {info[0]}, lowest eqbal ratio {info[1]:.4f}, closest bkr {info[2]:.4f}')
result = {**config_to_test, **{'lowest_eqbal_ratio': info[1], 'closest_bkr': info[2]}}
fdf, analysis = analyze_fills(fills, result, timestamps[0], timestamps[-1])
pprint.pprint(analysis)
fdf

In [ ]:
plot_fills(df, fdf)

In [ ]:
fdf.balance.plot()
fdf.equity.plot()

In [ ]:
# load external fills to inspect

fdf = pd.read_csv('backtests/binance/BTSUSDT/plots/2021-06-12T114030/fills.csv').set_index('trade_id')

In [ ]:
longs = fdf[fdf.type.str.contains('long')]
shrts = fdf[fdf.type.str.contains('shrt')]

In [ ]:
longs

In [ ]:
longs.tail(60)

In [ ]:
plot_fills(df, fdf)

In [ ]:
fdf.balance.plot()
fdf.equity.plot()